In [1]:
import os
import pandas as pd
import numpy as np
import time
from tqdm import notebook as nb
import requests
from bs4 import BeautifulSoup
import re
import os
import pickle
import glob

import matplotlib.pyplot as plt
from collections import Counter
import datetime

import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler



In [32]:
cols = ["登番","1号艇出走回数","1号艇1着回数","1号艇2着回数","1号艇3着回数","1号艇4着回数","1号艇5着回数","1号艇6着回数","2号艇出走回数","2号艇1着回数","2号艇2着回数","2号艇3着回数","2号艇4着回数","2号艇5着回数","2号艇6着回数","3号艇出走回数","3号艇1着回数","3号艇2着回数","3号艇3着回数","3号艇4着回数","3号艇5着回数","3号艇6着回数","4号艇出走回数","4号艇1着回数","4号艇2着回数","4号艇3着回数","4号艇4着回数","4号艇5着回数","4号艇6着回数","5号艇出走回数","5号艇1着回数","5号艇2着回数","5号艇3着回数","5号艇4着回数","5号艇5着回数","5号艇6着回数","6号艇出走回数","6号艇1着回数","6号艇2着回数","6号艇3着回数","6号艇4着回数","6号艇5着回数","6号艇6着回数"]
players = pd.DataFrame(columns=cols)
players = players.astype("int64")
result_root = "/mounted/data/results/csv"
file_list = sorted(glob.glob('/mounted/data/results/csv/k1[1-3]*.csv'))
for f in file_list:
    print(f)
    result = pd.read_csv(f)
    for index, row in result.iterrows():
        if ~players["登番"].isin([row["登番"]]).any():
            empty = 43*[0]
            tmp_list = [empty]
            tmp = pd.DataFrame(data=tmp_list, columns=cols)
            tmp["登番"] = row["登番"]
            players = pd.concat([players, tmp], ignore_index=True, axis=0)
        if (row["艇"] >= 1) & (row["艇"] <= 6):
            if type(row["着"]) is str:
                if row["着"].isdecimal():
                    if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] + 1
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] + 1
            else:
                if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                    players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] + 1
                    players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] + 1
players 

/mounted/data/results/csv/k110101.csv
/mounted/data/results/csv/k110102.csv
/mounted/data/results/csv/k110103.csv
/mounted/data/results/csv/k110104.csv
/mounted/data/results/csv/k110105.csv
/mounted/data/results/csv/k110106.csv
/mounted/data/results/csv/k110107.csv
/mounted/data/results/csv/k110108.csv
/mounted/data/results/csv/k110109.csv
/mounted/data/results/csv/k110110.csv
/mounted/data/results/csv/k110111.csv
/mounted/data/results/csv/k110112.csv
/mounted/data/results/csv/k110113.csv
/mounted/data/results/csv/k110114.csv
/mounted/data/results/csv/k110115.csv
/mounted/data/results/csv/k110116.csv
/mounted/data/results/csv/k110117.csv
/mounted/data/results/csv/k110118.csv
/mounted/data/results/csv/k110119.csv
/mounted/data/results/csv/k110120.csv
/mounted/data/results/csv/k110121.csv
/mounted/data/results/csv/k110122.csv
/mounted/data/results/csv/k110123.csv
/mounted/data/results/csv/k110124.csv
/mounted/data/results/csv/k110125.csv
/mounted/data/results/csv/k110126.csv
/mounted/dat

,登番,1号艇出走回数,1号艇1着回数,1号艇2着回数,1号艇3着回数,1号艇4着回数,1号艇5着回数,1号艇6着回数,2号艇出走回数,2号艇1着回数,...,5号艇4着回数,5号艇5着回数,5号艇6着回数,6号艇出走回数,6号艇1着回数,6号艇2着回数,6号艇3着回数,6号艇4着回数,6号艇5着回数,6号艇6着回数
0,4503,161,92,27,13,13,8,8,141,31,...,18,18,12,112,13,18,27,21,19,14
1,3471,105,31,32,14,8,7,13,111,19,...,28,22,17,100,3,15,16,23,20,23
2,2223,79,8,9,16,12,16,18,85,4,...,15,18,26,88,2,3,9,20,29,25
3,3650,90,34,18,9,8,7,14,102,14,...,12,22,33,96,8,9,10,11,18,40
4,3678,86,15,13,9,14,19,16,108,7,...,20,31,37,89,0,7,10,19,29,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672,4778,0,0,0,0,0,0,0,0,0,...,0,3,3,8,0,0,0,0,3,5
1673,4803,0,0,0,0,0,0,0,0,0,...,1,2,5,5,0,0,1,1,2,1
1674,4777,0,0,0,0,0,0,0,0,0,...,0,0,5,6,0,0,0,0,0,6
1675,4799,0,0,0,0,0,0,0,0,0,...,0,1,4,10,0,0,0,1,3,6


In [115]:
players.to_csv("/mounted/tobe/players/20131231.csv")

In [114]:
cols = ["登番","1号艇出走回数","1号艇1着回数","1号艇2着回数","1号艇3着回数","1号艇4着回数","1号艇5着回数","1号艇6着回数","2号艇出走回数","2号艇1着回数","2号艇2着回数","2号艇3着回数","2号艇4着回数","2号艇5着回数","2号艇6着回数","3号艇出走回数","3号艇1着回数","3号艇2着回数","3号艇3着回数","3号艇4着回数","3号艇5着回数","3号艇6着回数","4号艇出走回数","4号艇1着回数","4号艇2着回数","4号艇3着回数","4号艇4着回数","4号艇5着回数","4号艇6着回数","5号艇出走回数","5号艇1着回数","5号艇2着回数","5号艇3着回数","5号艇4着回数","5号艇5着回数","5号艇6着回数","6号艇出走回数","6号艇1着回数","6号艇2着回数","6号艇3着回数","6号艇4着回数","6号艇5着回数","6号艇6着回数"]
file_list = sorted(glob.glob('/mounted/data/players/csv/fan*.csv'))
players = pd.read_csv("/mounted/tobe/players/20131231.csv", index_col=0)
players = players.dropna(how="any")
players = players.astype({'登番': int})
# for f in file_list:
    result = pd.read_csv(f)
    for index, row in result.iterrows():
        if ~players["登番"].isin([row["登番"]]).any():
            empty = 43*[0]
            tmp_list = [empty]
            tmp = pd.DataFrame(data=tmp_list, columns=cols)
            tmp["登番"] = row["登番"]
            players = pd.concat([players, tmp], ignore_index=True, axis=0)
# players[""]

In [2]:
players = pd.read_csv("/mounted/tobe/players/202307.csv", index_col=0)
race_place = {"桐生":1,"戸田":2,"江戸川":3,"平和島":4,"多摩川":5,"浜名湖":6,"蒲郡":7,"常滑":8,"津":9,"三国":10,"びわこ":11,"住之江":12,
              "尼崎":13,"鳴門":14,"丸亀":15,"児島":16,"宮島":17,"徳山":18,"下関":19,"若松":20,"芦屋":21,"福岡":22,"唐津":23,"大村":24}
arrival = ["01","02","03","04","05","06"]
cols = ["レースID","123",'1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率']
col2 = ["登番","1号艇出走回数","1号艇1着回数","1号艇2着回数","1号艇3着回数","1号艇4着回数","1号艇5着回数","1号艇6着回数","2号艇出走回数","2号艇1着回数","2号艇2着回数","2号艇3着回数","2号艇4着回数","2号艇5着回数","2号艇6着回数","3号艇出走回数","3号艇1着回数","3号艇2着回数","3号艇3着回数","3号艇4着回数","3号艇5着回数","3号艇6着回数","4号艇出走回数","4号艇1着回数","4号艇2着回数","4号艇3着回数","4号艇4着回数","4号艇5着回数","4号艇6着回数","5号艇出走回数","5号艇1着回数","5号艇2着回数","5号艇3着回数","5号艇4着回数","5号艇5着回数","5号艇6着回数","6号艇出走回数","6号艇1着回数","6号艇2着回数","6号艇3着回数","6号艇4着回数","6号艇5着回数","6号艇6着回数"]
file_list = [p for p in glob.glob('/mounted/data/results/csv/*') if re.search('k2(00[8-9]|01|[1-3])',p)]
count = 0
for year in range(2308,2312):
    dataset = pd.DataFrame(columns=cols)
    files = sorted(glob.glob('/mounted/tobe/dataset_honban/k'+str(year)+'*.csv'))
    print(year)
    for f in files:
        print(f)
        race = pd.read_csv(f)
        race["レースID"] = [str(v["日付"]) + str(race_place[v["開催地"]]).zfill(2) + str(v["ラウンド名"]).zfill(2) for i, v in race.iterrows()]
        race = race.reset_index(drop=True)
        race = pd.merge(race, players, how="left", on = "登番")
        for ide in race["レースID"].unique():
            flag = True
            if race.loc[(race["レースID"] == ide),"着"].isin(arrival).all():
                merged = pd.DataFrame(data=[[0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],columns=cols)
                merged.loc[0,"レースID"] = ide
                if (race.loc[(race["レースID"] == ide) & (race["艇"] == 1),"着"]=="01").all() & (race.loc[(race["レースID"] == ide) & (race["艇"] == 2),"着"]=="02").all() & (race.loc[(race["レースID"] == ide) & (race["艇"] == 3),"着"]=="03").all():
                    merged.loc[0,"123"] = True
                else:
                    merged.loc[0,"123"] = False
                for i in range(1,7):
                    for j in range(1,4):
                        if (race.loc[(race["レースID"] == ide) & (race["艇"] == i),str(i)+"号艇出走回数"] < 10).all():
                            flag = False
                        merged.loc[0,str(i)+"号艇"+str(j)+"着率"] = (race.loc[(race["レースID"] == ide) & (race["艇"] == i),str(i)+"号艇"+str(j)+"着回数"] / race.loc[(race["レースID"] == ide) & (race["艇"] == i),str(i)+"号艇出走回数"]).iloc[-1]
                if flag == True:
                    dataset = pd.concat([dataset, merged], ignore_index=True, axis=0)
        race = pd.read_csv(f)
        for index, row in race.iterrows():
            if ~players["登番"].isin([row["登番"]]).any():
                empty = 43*[0]
                tmp_list = [empty]
                tmp = pd.DataFrame(data=tmp_list, columns=col2)
                tmp["登番"] = row["登番"]
                players = pd.concat([players, tmp], ignore_index=True, axis=0)
            if (row["艇"] >= 1) & (row["艇"] <= 6):
                if type(row["着"]) is str:
                    if row["着"].isdecimal():
                        if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                            players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] + 1
                            players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] + 1
                else:
                    if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] + 1
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] + 1
        exc = pd.read_csv(file_list[count])
        for index, row in exc.iterrows():
            if (row["艇"] >= 1) & (row["艇"] <= 6):
                if type(row["着"]) is str:
                    if row["着"].isdecimal():
                        if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                            players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] - 1
                            players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] - 1
                else:
                    if (int(row["着"]) <= 6) & (int(row["着"]) >= 1):
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇" + str(int(row["着"])) + "着回数"] - 1
                        players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] = players.loc[players["登番"].isin([row["登番"]]),str(row["艇"]) + "号艇出走回数"] - 1
        count = count + 1
    players.to_csv("/mounted/tobe/players/20"+str(year)+".csv")
    dataset.to_csv("/mounted/tobe/dataset/dataset"+str(year)+".csv")


2308
/mounted/tobe/dataset_honban/k230801.csv
/mounted/tobe/dataset_honban/k230802.csv
/mounted/tobe/dataset_honban/k230803.csv
/mounted/tobe/dataset_honban/k230804.csv
/mounted/tobe/dataset_honban/k230805.csv
/mounted/tobe/dataset_honban/k230806.csv
/mounted/tobe/dataset_honban/k230807.csv
/mounted/tobe/dataset_honban/k230808.csv
/mounted/tobe/dataset_honban/k230809.csv
/mounted/tobe/dataset_honban/k230810.csv
/mounted/tobe/dataset_honban/k230811.csv
/mounted/tobe/dataset_honban/k230812.csv
/mounted/tobe/dataset_honban/k230813.csv
/mounted/tobe/dataset_honban/k230814.csv
/mounted/tobe/dataset_honban/k230815.csv
/mounted/tobe/dataset_honban/k230816.csv
/mounted/tobe/dataset_honban/k230817.csv
/mounted/tobe/dataset_honban/k230818.csv
/mounted/tobe/dataset_honban/k230819.csv
/mounted/tobe/dataset_honban/k230820.csv
/mounted/tobe/dataset_honban/k230821.csv
/mounted/tobe/dataset_honban/k230822.csv
/mounted/tobe/dataset_honban/k230823.csv
/mounted/tobe/dataset_honban/k230824.csv
/mounted/to

In [16]:
players_root = "/mounted/data/players/csv/"
players_list = ["fan1704.csv","fan1710.csv","fan1804.csv","fan1810.csv","fan1904.csv"]
h = ''
data = pd.read_csv(players_root + "fan1910.csv")
data = data[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
for j,player in enumerate(players_list):
    tmp = pd.read_csv(players_root + player)
    tmp = tmp[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
    for i in range(1,7):
        tmp = tmp.rename(columns={f'{i}コース進入回数':f'{i}コース進入回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース1着回数':f'{i}コース1着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース2着回数':f'{i}コース2着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース3着回数':f'{i}コース3着回数_{j}'})
    data = pd.merge(data,tmp,how="left",on="登番")
    data.dropna(how='all', inplace=True)
    data = data.fillna(0)
    for i in range(1,7):
        data[f'{i}コース進入回数'] = data[f'{i}コース進入回数'] + data[f'{i}コース進入回数_{j}']
        data[f'{i}コース1着回数'] = data[f'{i}コース1着回数'] + data[f'{i}コース1着回数_{j}']
        data[f'{i}コース2着回数'] = data[f'{i}コース2着回数'] + data[f'{i}コース2着回数_{j}']
        data[f'{i}コース3着回数'] = data[f'{i}コース3着回数'] + data[f'{i}コース3着回数_{j}']
        data = data.drop(f'{i}コース進入回数_{j}', axis=1)
        data = data.drop(f'{i}コース1着回数_{j}', axis=1)
        data = data.drop(f'{i}コース2着回数_{j}', axis=1)
        data = data.drop(f'{i}コース3着回数_{j}', axis=1)
for i in range(1,7):
    data = data[data[f'{i}コース進入回数'] > 10]
for i in range(1,7):
    data[f'{i}コース1着率'] = (data[f'{i}コース1着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース2着率'] = (data[f'{i}コース2着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース3着率'] = (data[f'{i}コース3着回数'] / data[f'{i}コース進入回数']) * 100
    data = data.drop(f'{i}コース進入回数', axis=1)
    data = data.drop(f'{i}コース1着回数', axis=1)
    data = data.drop(f'{i}コース2着回数', axis=1)
    data = data.drop(f'{i}コース3着回数', axis=1)
data = data.fillna(0)
# data['1コース進入回数'] = data['1コース進入回数'] + data['1コース進入回数_0']
data
# print(players_list)

,登番,1コース1着率,1コース2着率,1コース3着率,2コース1着率,2コース2着率,2コース3着率,3コース1着率,3コース2着率,3コース3着率,4コース1着率,4コース2着率,4コース3着率,5コース1着率,5コース2着率,5コース3着率,6コース1着率,6コース2着率,6コース3着率
0,2014.0,25.000000,13.541667,23.958333,11.818182,22.727273,21.818182,7.216495,12.371134,20.618557,9.782609,17.391304,30.434783,6.542056,13.084112,28.971963,1.136364,9.090909,12.500000
1,2538.0,27.160494,14.814815,12.345679,3.759398,20.300752,24.060150,7.017544,12.280702,18.421053,1.010101,9.090909,17.171717,1.388889,6.944444,13.888889,0.000000,1.562500,15.625000
2,2787.0,43.434343,25.252525,11.111111,15.841584,24.752475,25.742574,8.943089,23.577236,15.447154,5.357143,8.928571,25.892857,0.000000,10.204082,14.285714,0.000000,0.000000,10.714286
3,2841.0,24.000000,36.000000,18.000000,5.000000,26.666667,20.000000,7.547170,21.698113,20.754717,9.160305,15.267176,29.770992,1.785714,9.821429,19.642857,0.000000,5.660377,9.433962
4,2844.0,30.000000,21.111111,11.111111,13.402062,26.804124,20.618557,5.769231,12.500000,20.192308,5.714286,14.285714,23.809524,1.075269,5.376344,6.451613,1.851852,7.407407,5.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,4988.0,11.111111,5.555556,11.111111,3.571429,7.142857,17.857143,5.882353,5.882353,17.647059,3.333333,3.333333,10.000000,1.538462,9.230769,1.538462,0.546448,1.092896,3.825137
1478,4991.0,66.666667,16.666667,5.555556,3.571429,32.142857,14.285714,0.000000,12.500000,33.333333,5.714286,14.285714,14.285714,2.040816,15.306122,22.448980,2.542373,5.508475,13.559322
1489,5002.0,28.571429,9.523810,9.523810,11.764706,17.647059,17.647059,2.941176,20.588235,23.529412,5.555556,8.333333,22.222222,6.451613,8.064516,8.064516,2.040816,1.360544,4.761905
1492,5005.0,31.250000,6.250000,12.500000,0.000000,20.689655,3.448276,0.000000,4.000000,20.000000,7.894737,2.631579,10.526316,4.761905,7.142857,9.523810,0.000000,0.724638,5.797101


In [17]:
file_list = sorted(glob.glob('/mounted/data/results/csv/k20*.csv'))
race_place = {"桐生":1,"戸田":2,"江戸川":3,"平和島":4,"多摩川":5,"浜名湖":6,"蒲郡":7,"常滑":8,"津":9,"三国":10,"びわこ":11,"住之江":12,
              "尼崎":13,"鳴門":14,"丸亀":15,"児島":16,"宮島":17,"徳山":18,"下関":19,"若松":20,"芦屋":21,"福岡":22,"唐津":23,"大村":24}
cols = ['1号艇着順','1号艇登番','1号艇1着率','1号艇2着率','1号艇3着率','2号艇着順','2号艇登番','2号艇1着率','2号艇2着率','2号艇3着率','3号艇着順','3号艇登番','3号艇1着率','3号艇2着率','3号艇3着率','4号艇着順','4号艇登番','4号艇1着率','4号艇2着率','4号艇3着率','5号艇着順','5号艇登番','5号艇1着率','5号艇2着率','5号艇3着率','6号艇着順','6号艇登番','6号艇1着率','6号艇2着率','6号艇3着率']
dataset = pd.DataFrame(columns=cols)
for file in file_list:
    merged = pd.DataFrame()
    results = pd.read_csv(file)
    results["レースID"] = [str(v["日付"]) + str(race_place[v["開催地"]]).zfill(2) + str(v["ラウンド名"]).zfill(2) for i, v in results.iterrows()]
    results = results.reset_index(drop=True)
    results = pd.merge(results, data, how="left", on = "登番")
    results.isnull().sum()
    merged = merged.reset_index(drop=True)
    for i in range(0,6):
        merged.insert(0,f'{6-i}号艇3着率',results[results['艇']==6-i][f'{6-i}コース3着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇2着率',results[results['艇']==6-i][f'{6-i}コース2着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇1着率',results[results['艇']==6-i][f'{6-i}コース1着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇登番',results[results['艇']==6-i]['登番'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇着順',results[results['艇']==6-i]['着'].reset_index(drop=True))
    merged.insert(0,'1着',results[results['着']=='01']['艇'].reset_index(drop=True))
    merged.insert(0,"レースID",results[results["着"]=="01"]['レースID'].reset_index(drop=True))
    # merged.insert(0,'2着',results[results['着']=='02']['艇'].reset_index(drop=True))
    # merged.insert(0,'3着',results[results['着']=='03']['艇'].reset_index(drop=True))
    # merged['123着'] = merged['1着'].astype(int).astype(str) + merged['2着'].astype(int).astype(str) + merged['3着'].astype(int).astype(str)
    dataset = pd.concat([dataset,merged])
dataset = dataset.reset_index(drop=True)
dataset
# dataset
# dataset.to_csv('/mounted/dataset/dataset01/dataset01.csv', index=False)
#  & (dataset['2号艇1着率'] < 20) & (dataset['3号艇1着率'] < 20) & (dataset['4号艇1着率'] < 20) & (dataset['5号艇1着率'] < 20)

,1号艇着順,1号艇登番,1号艇1着率,1号艇2着率,1号艇3着率,2号艇着順,2号艇登番,2号艇1着率,2号艇2着率,2号艇3着率,...,5号艇1着率,5号艇2着率,5号艇3着率,6号艇着順,6号艇登番,6号艇1着率,6号艇2着率,6号艇3着率,レースID,1着
0,02,4164,32.857143,20.000000,10.000000,03,2958,8.620690,19.827586,20.689655,...,3.658537,6.097561,10.975610,06,5009,NaN,NaN,NaN,202001012401,4.0
1,02,4005,28.169014,15.492958,15.492958,04,3843,10.687023,27.480916,16.793893,...,1.980198,8.910891,10.891089,05,3151,1.666667,3.333333,15.000000,202001012402,3.0
2,01,3501,58.064516,14.516129,9.677419,05,3964,3.773585,19.811321,23.584906,...,6.666667,13.333333,19.166667,06,4603,0.000000,6.194690,11.504425,202001012403,1.0
3,01,3527,62.500000,16.304348,8.695652,04,3936,16.071429,17.857143,17.857143,...,1.754386,8.771930,18.421053,02,4041,0.000000,4.000000,21.333333,202001012404,1.0
4,02,4553,53.608247,15.463918,10.309278,04,4966,0.000000,22.727273,13.636364,...,12.820513,30.769231,12.820513,06,2912,0.000000,5.434783,13.043478,202001012405,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54878,01,3347,55.000000,21.666667,10.000000,02,3748,1.136364,18.181818,19.318182,...,8.163265,12.244898,21.428571,04,3647,1.052632,4.210526,15.789474,202012310108,1.0
54879,01,5051,NaN,NaN,NaN,03,4875,6.329114,31.645570,24.050633,...,6.122449,13.265306,14.285714,02,4863,0.877193,2.631579,7.017544,202012310109,1.0
54880,01,4083,73.619632,11.042945,4.907975,02,3952,30.281690,26.760563,16.901408,...,2.307692,11.538462,21.538462,05,4637,3.092784,4.123711,11.340206,202012310110,1.0
54881,01,3517,63.063063,19.819820,6.306306,03,3463,13.934426,31.147541,13.934426,...,6.451613,16.935484,25.000000,05,3772,0.000000,1.449275,11.594203,202012310111,1.0


In [18]:
players_root = "/mounted/data/players/csv/"
players_list = ["fan2010.csv","fan2104.csv","fan2110.csv","fan2204.csv","fan2210.csv"]
h = ''
data = pd.read_csv(players_root + "fan2304.csv")
data = data[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
for j,player in enumerate(players_list):
    tmp = pd.read_csv(players_root + player)
    tmp = tmp[['登番','1コース進入回数','1コース1着回数','1コース2着回数','1コース3着回数','2コース進入回数','2コース1着回数','2コース2着回数','2コース3着回数','3コース進入回数','3コース1着回数','3コース2着回数','3コース3着回数','4コース進入回数','4コース1着回数','4コース2着回数','4コース3着回数','5コース進入回数','5コース1着回数','5コース2着回数','5コース3着回数','6コース進入回数','6コース1着回数','6コース2着回数','6コース3着回数']]
    for i in range(1,7):
        tmp = tmp.rename(columns={f'{i}コース進入回数':f'{i}コース進入回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース1着回数':f'{i}コース1着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース2着回数':f'{i}コース2着回数_{j}'})
        tmp = tmp.rename(columns={f'{i}コース3着回数':f'{i}コース3着回数_{j}'})
    data = pd.merge(data,tmp,how="left",on="登番")
    data.dropna(how='all', inplace=True)
    data = data.fillna(0)
    for i in range(1,7):
        data[f'{i}コース進入回数'] = data[f'{i}コース進入回数'] + data[f'{i}コース進入回数_{j}']
        data[f'{i}コース1着回数'] = data[f'{i}コース1着回数'] + data[f'{i}コース1着回数_{j}']
        data[f'{i}コース2着回数'] = data[f'{i}コース2着回数'] + data[f'{i}コース2着回数_{j}']
        data[f'{i}コース3着回数'] = data[f'{i}コース3着回数'] + data[f'{i}コース3着回数_{j}']
        data = data.drop(f'{i}コース進入回数_{j}', axis=1)
        data = data.drop(f'{i}コース1着回数_{j}', axis=1)
        data = data.drop(f'{i}コース2着回数_{j}', axis=1)
        data = data.drop(f'{i}コース3着回数_{j}', axis=1)
for i in range(1,7):
    data = data[data[f'{i}コース進入回数'] > 10]
for i in range(1,7):
    data[f'{i}コース1着率'] = (data[f'{i}コース1着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース2着率'] = (data[f'{i}コース2着回数'] / data[f'{i}コース進入回数']) * 100
    data[f'{i}コース3着率'] = (data[f'{i}コース3着回数'] / data[f'{i}コース進入回数']) * 100
    data = data.drop(f'{i}コース進入回数', axis=1)
    data = data.drop(f'{i}コース1着回数', axis=1)
    data = data.drop(f'{i}コース2着回数', axis=1)
    data = data.drop(f'{i}コース3着回数', axis=1)
data = data.fillna(0)
# data['1コース進入回数'] = data['1コース進入回数'] + data['1コース進入回数_0']
data
# print(players_list)

,登番,1コース1着率,1コース2着率,1コース3着率,2コース1着率,2コース2着率,2コース3着率,3コース1着率,3コース2着率,3コース3着率,4コース1着率,4コース2着率,4コース3着率,5コース1着率,5コース2着率,5コース3着率,6コース1着率,6コース2着率,6コース3着率
0,2014.0,31.250000,22.321429,17.857143,7.964602,25.663717,15.929204,13.333333,13.333333,15.238095,7.920792,13.861386,22.772277,1.769912,7.079646,15.929204,2.325581,6.976744,10.465116
1,2538.0,17.045455,19.318182,18.181818,5.691057,15.447154,21.951220,4.081633,16.326531,18.367347,2.325581,13.953488,18.604651,0.000000,6.024096,22.891566,2.040816,2.040816,22.448980
2,2878.0,39.860140,27.972028,16.783217,11.363636,23.484848,21.969697,11.904762,16.666667,29.365079,15.107914,15.827338,26.618705,5.600000,7.200000,24.000000,1.587302,11.111111,19.047619
3,2903.0,31.764706,20.000000,11.764706,5.847953,21.637427,25.146199,4.950495,14.851485,28.712871,5.454545,14.545455,19.090909,0.000000,8.695652,10.144928,0.000000,2.941176,14.705882
4,2930.0,44.155844,22.077922,15.584416,12.048193,25.301205,21.686747,4.761905,21.428571,23.809524,2.777778,12.500000,16.666667,0.000000,5.970149,17.910448,0.000000,3.703704,7.407407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1490,5197.0,23.076923,23.076923,7.692308,8.695652,26.086957,8.695652,9.523810,19.047619,14.285714,4.000000,4.000000,16.000000,1.388889,5.555556,6.944444,0.571429,2.285714,5.714286
1492,5199.0,25.000000,18.750000,12.500000,3.448276,6.896552,20.689655,3.448276,17.241379,13.793103,0.000000,5.263158,5.263158,7.017544,10.526316,7.017544,0.558659,0.000000,3.910615
1494,5201.0,21.428571,21.428571,7.142857,0.000000,19.047619,33.333333,20.000000,6.666667,13.333333,8.333333,8.333333,13.333333,0.000000,2.531646,15.189873,0.000000,0.671141,7.382550
1514,5221.0,57.142857,0.000000,14.285714,11.111111,27.777778,11.111111,4.347826,13.043478,13.043478,8.823529,17.647059,20.588235,2.173913,2.173913,15.217391,0.714286,5.000000,7.142857


In [43]:
file_list = sorted(glob.glob('/mounted/data/results/csv/k2308*.csv'))
race_place = {"桐生":1,"戸田":2,"江戸川":3,"平和島":4,"多摩川":5,"浜名湖":6,"蒲郡":7,"常滑":8,"津":9,"三国":10,"びわこ":11,"住之江":12,
              "尼崎":13,"鳴門":14,"丸亀":15,"児島":16,"宮島":17,"徳山":18,"下関":19,"若松":20,"芦屋":21,"福岡":22,"唐津":23,"大村":24}
cols = ['1号艇着順','1号艇登番','1号艇1着率','1号艇2着率','1号艇3着率','2号艇着順','2号艇登番','2号艇1着率','2号艇2着率','2号艇3着率','3号艇着順','3号艇登番','3号艇1着率','3号艇2着率','3号艇3着率','4号艇着順','4号艇登番','4号艇1着率','4号艇2着率','4号艇3着率','5号艇着順','5号艇登番','5号艇1着率','5号艇2着率','5号艇3着率','6号艇着順','6号艇登番','6号艇1着率','6号艇2着率','6号艇3着率']
dataset_test = pd.DataFrame(columns=cols)
for file in file_list:
    merged = pd.DataFrame()
    results = pd.read_csv(file)
    results["レースID"] = [str(v["日付"]) + str(race_place[v["開催地"]]).zfill(2) + str(v["ラウンド名"]).zfill(2) for i, v in results.iterrows()]
    results = results.reset_index(drop=True)
    results = pd.merge(results, data, how="left", on = "登番")
    results.isnull().sum()
    merged = merged.reset_index(drop=True)
    for i in range(0,6):
        merged.insert(0,f'{6-i}号艇3着率',results[results['艇']==6-i][f'{6-i}コース3着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇2着率',results[results['艇']==6-i][f'{6-i}コース2着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇1着率',results[results['艇']==6-i][f'{6-i}コース1着率'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇登番',results[results['艇']==6-i]['登番'].reset_index(drop=True))
        merged.insert(0,f'{6-i}号艇着順',results[results['艇']==6-i]['着'].reset_index(drop=True))
    merged.insert(0,'1着',results[results['着']=='01']['艇'].reset_index(drop=True))
    merged.insert(0,"レースID",results[results["着"]=="01"]['レースID'].reset_index(drop=True))
    merged.insert(0,"日付",results[results["着"]=="01"]['日付'].reset_index(drop=True))
    merged.insert(0,"開催地",results[results["着"]=="01"]['開催地'].reset_index(drop=True))
    merged.insert(0,"ラウンド名",results[results["着"]=="01"]['ラウンド名'].reset_index(drop=True))
    
    # merged.insert(0,'2着',results[results['着']=='02']['艇'].reset_index(drop=True))
    # merged.insert(0,'3着',results[results['着']=='03']['艇'].reset_index(drop=True))
    # merged['123着'] = merged['1着'].astype(int).astype(str) + merged['2着'].astype(int).astype(str) + merged['3着'].astype(int).astype(str)
    dataset_test = pd.concat([dataset_test,merged])
dataset_test = dataset_test.reset_index(drop=True)
dataset_test
# dataset
# dataset.to_csv('/mounted/dataset/dataset01/dataset01.csv', index=False)
#  & (dataset['2号艇1着率'] < 20) & (dataset['3号艇1着率'] < 20) & (dataset['4号艇1着率'] < 20) & (dataset['5号艇1着率'] < 20)

,1号艇着順,1号艇登番,1号艇1着率,1号艇2着率,1号艇3着率,2号艇着順,2号艇登番,2号艇1着率,2号艇2着率,2号艇3着率,...,6号艇着順,6号艇登番,6号艇1着率,6号艇2着率,6号艇3着率,ラウンド名,開催地,日付,レースID,1着
0,03,3805,58.677686,23.140496,8.264463,02,4031,7.142857,29.591837,17.346939,...,01,5258,NaN,NaN,NaN,1.0,唐津,20230801.0,202308012301,6.0
1,01,4983,69.590643,8.771930,7.017544,02,3681,25.225225,27.027027,20.720721,...,03,5115,2.352941,2.941176,6.470588,2.0,唐津,20230801.0,202308012302,1.0
2,01,4959,65.562914,14.569536,9.271523,02,3880,13.178295,29.457364,17.054264,...,04,4417,2.061856,9.278351,20.618557,3.0,唐津,20230801.0,202308012303,1.0
3,01,5106,50.000000,15.000000,15.000000,04,4692,15.094340,25.471698,18.867925,...,05,5159,1.156069,0.578035,3.468208,4.0,唐津,20230801.0,202308012304,1.0
4,01,3761,47.747748,25.225225,11.711712,03,4415,25.925926,26.666667,14.074074,...,05,5177,NaN,NaN,NaN,5.0,唐津,20230801.0,202308012305,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,01,5141,59.322034,11.864407,13.559322,05,5133,8.510638,12.765957,19.148936,...,04,5150,NaN,NaN,NaN,8.0,桐生,20230816.0,202308160108,1.0
2726,01,4539,73.793103,12.413793,6.206897,02,4086,10.576923,32.692308,25.000000,...,06,3532,3.571429,12.500000,17.857143,9.0,桐生,20230816.0,202308160109,1.0
2727,01,3659,47.286822,33.333333,6.976744,05,3475,22.727273,25.757576,15.909091,...,03,3655,4.705882,8.235294,24.705882,10.0,桐生,20230816.0,202308160110,1.0
2728,01,4362,77.157360,9.644670,5.076142,05,3406,13.846154,23.076923,26.153846,...,04,5141,3.508772,7.602339,9.941520,11.0,桐生,20230816.0,202308160111,1.0


In [44]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x - x.mean()) / x.std(), axis=0)

In [35]:
# dataset1 = dataset[(dataset['1号艇1着率'] > 60) & (dataset['2号艇1着率'] < 10) & (dataset['3号艇1着率'] < 10) & (dataset['4号艇1着率'] < 10) & (dataset['5号艇1着率'] < 10)]
# # dataset1['2号艇2着率'].hist(bins = 12)
# # dataset1 = dataset1[(dataset1['2号艇2着率'] > 25)]
# # dataset1['6号艇2着率'].hist(bins = 20)
# dataset1 = dataset1[dataset1['123着'] == '156']
# print(dataset['123着'].value_counts())
conditions = [
    (dataset['1号艇着順'] == "01") & (dataset["2号艇着順"] == "02") & (dataset["3号艇着順"] == "03")
]
choices = [1]
dataset['123'] = np.select(conditions, choices, default=0)
dataset.dropna(how='any',inplace=True)
dataset['1着'] = dataset['1着'] - 1
dataset=dataset.reset_index(drop=True)
dataset
# dataset.describe()
# print(dataset1['3号艇着順'].value_counts())
# print(dataset1['4号艇着順'].value_counts())
# print(dataset1['5号艇着順'].value_counts())
# print(dataset1['6号艇着順'].value_counts())

,1号艇着順,1号艇登番,1号艇1着率,1号艇2着率,1号艇3着率,2号艇着順,2号艇登番,2号艇1着率,2号艇2着率,2号艇3着率,...,5号艇2着率,5号艇3着率,6号艇着順,6号艇登番,6号艇1着率,6号艇2着率,6号艇3着率,レースID,1着,123
0,02,4005,28.169014,15.492958,15.492958,04,3843,10.687023,27.480916,16.793893,...,8.910891,10.891089,05,3151,1.666667,3.333333,15.000000,202001012402,1.0,0
1,01,3501,58.064516,14.516129,9.677419,05,3964,3.773585,19.811321,23.584906,...,13.333333,19.166667,06,4603,0.000000,6.194690,11.504425,202001012403,-1.0,0
2,01,3527,62.500000,16.304348,8.695652,04,3936,16.071429,17.857143,17.857143,...,8.771930,18.421053,02,4041,0.000000,4.000000,21.333333,202001012404,-1.0,0
3,02,4553,53.608247,15.463918,10.309278,04,4966,0.000000,22.727273,13.636364,...,30.769231,12.820513,06,2912,0.000000,5.434783,13.043478,202001012405,3.0,0
4,01,4259,72.067039,16.201117,4.469274,02,3773,17.647059,14.705882,18.382353,...,3.809524,17.142857,06,3151,1.666667,3.333333,15.000000,202001012407,-1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30995,02,4535,60.655738,20.491803,9.016393,01,4787,28.455285,26.829268,10.569106,...,7.547170,15.094340,06,4496,5.681818,10.227273,23.863636,202012310212,0.0,0
30996,01,3952,62.621359,14.563107,8.252427,05,4139,17.886179,23.577236,14.634146,...,13.913043,26.086957,04,4103,0.000000,5.617978,7.865169,202012310106,-1.0,0
30997,01,4083,73.619632,11.042945,4.907975,02,3952,30.281690,26.760563,16.901408,...,11.538462,21.538462,05,4637,3.092784,4.123711,11.340206,202012310110,-1.0,0
30998,01,3517,63.063063,19.819820,6.306306,03,3463,13.934426,31.147541,13.934426,...,16.935484,25.000000,05,3772,0.000000,1.449275,11.594203,202012310111,-1.0,0


In [53]:
# dataset1 = dataset[(dataset['1号艇1着率'] > 60) & (dataset['2号艇1着率'] < 10) & (dataset['3号艇1着率'] < 10) & (dataset['4号艇1着率'] < 10) & (dataset['5号艇1着率'] < 10)]
# # dataset1['2号艇2着率'].hist(bins = 12)
# # dataset1 = dataset1[(dataset1['2号艇2着率'] > 25)]
# # dataset1['6号艇2着率'].hist(bins = 20)
# dataset1 = dataset1[dataset1['123着'] == '156']
# print(dataset['123着'].value_counts())
conditions = [
    (dataset_test['1号艇着順'] == "01") & (dataset_test["2号艇着順"] == "02") & (dataset_test["3号艇着順"] == "03")
]
choices = [1]
dataset_test['123'] = np.select(conditions, choices, default=0)
dataset_test.dropna(how='any',inplace=True)
dataset_test['1着'] = dataset_test['1着'] - 1
dataset_test=dataset_test.reset_index(drop=True)
dataset_test
# dataset.describe()
# print(dataset1['3号艇着順'].value_counts())
# print(dataset1['4号艇着順'].value_counts())
# print(dataset1['5号艇着順'].value_counts())
# print(dataset1['6号艇着順'].value_counts())

,1号艇着順,1号艇登番,1号艇1着率,1号艇2着率,1号艇3着率,2号艇着順,2号艇登番,2号艇1着率,2号艇2着率,2号艇3着率,...,6号艇登番,6号艇1着率,6号艇2着率,6号艇3着率,ラウンド名,開催地,日付,レースID,1着,123
0,01,4983,69.590643,8.771930,7.017544,02,3681,25.225225,27.027027,20.720721,...,5115,2.352941,2.941176,6.470588,2.0,唐津,20230801.0,202308012302,0.0,0
1,01,5106,50.000000,15.000000,15.000000,04,4692,15.094340,25.471698,18.867925,...,5159,1.156069,0.578035,3.468208,4.0,唐津,20230801.0,202308012304,0.0,0
2,01,4740,29.629630,19.753086,12.345679,05,4417,9.183673,19.387755,27.551020,...,5044,1.063830,4.255319,12.765957,7.0,唐津,20230801.0,202308012307,0.0,0
3,01,4837,38.461538,12.307692,15.384615,06,5159,3.846154,3.846154,3.846154,...,3880,0.000000,6.250000,15.625000,8.0,唐津,20230801.0,202308012308,0.0,0
4,02,4258,43.023256,15.116279,10.465116,06,3951,14.953271,24.299065,24.299065,...,5082,0.724638,6.521739,10.144928,9.0,唐津,20230801.0,202308012309,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617,01,4851,73.142857,12.000000,7.428571,06,3850,12.213740,23.664122,23.664122,...,3652,1.149425,1.149425,6.896552,6.0,桐生,20230816.0,202308160106,0.0,0
1618,01,4188,75.000000,15.476190,1.785714,02,4491,15.887850,11.214953,20.560748,...,4037,2.702703,5.405405,17.117117,7.0,桐生,20230816.0,202308160107,0.0,0
1619,01,4539,73.793103,12.413793,6.206897,02,4086,10.576923,32.692308,25.000000,...,3532,3.571429,12.500000,17.857143,9.0,桐生,20230816.0,202308160109,0.0,1
1620,01,4362,77.157360,9.644670,5.076142,05,3406,13.846154,23.076923,26.153846,...,5141,3.508772,7.602339,9.941520,11.0,桐生,20230816.0,202308160111,0.0,0


In [115]:
print(dataset.dtypes)


1号艇着順      object
1号艇登番      object
1号艇1着率    float64
1号艇2着率    float64
1号艇3着率    float64
2号艇着順      object
2号艇登番      object
2号艇1着率    float64
2号艇2着率    float64
2号艇3着率    float64
3号艇着順      object
3号艇登番      object
3号艇1着率    float64
3号艇2着率    float64
3号艇3着率    float64
4号艇着順      object
4号艇登番      object
4号艇1着率    float64
4号艇2着率    float64
4号艇3着率    float64
5号艇着順      object
5号艇登番      object
5号艇1着率    float64
5号艇2着率    float64
5号艇3着率    float64
6号艇着順      object
6号艇登番      object
6号艇1着率    float64
6号艇2着率    float64
6号艇3着率    float64
1着        float64
123         int64
dtype: object


In [54]:

features_train = dataset[['1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率']]
features_train = mean_norm(features_train)
label_train = dataset['123'].astype(int)
features_test = dataset_test[['1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率']]
features_test = mean_norm(features_test)
label_test = dataset_test['123'].astype(int)
print(features_train.dtypes)

1号艇1着率    float64
1号艇2着率    float64
1号艇3着率    float64
2号艇1着率    float64
2号艇2着率    float64
2号艇3着率    float64
3号艇1着率    float64
3号艇2着率    float64
3号艇3着率    float64
4号艇1着率    float64
4号艇2着率    float64
4号艇3着率    float64
5号艇1着率    float64
5号艇2着率    float64
5号艇3着率    float64
6号艇1着率    float64
6号艇2着率    float64
6号艇3着率    float64
dtype: object


In [62]:
Linsvc = svm.LinearSVC(random_state=0, max_iter=3500)
Linsvc.fit(features_train,label_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(max_iter=3500, random_state=0)

In [37]:
label_pred_Linsvc = Linsvc.predict(features_test)
print(label_pred_Linsvc)
print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_Linsvc))
print('accuracy = ', Linsvc.score(features_test,label_test))

NameError: name 'Linsvc' is not defined

In [73]:
Kneighbor = KNeighborsClassifier(n_neighbors=1)
 
#K近傍法でデータを学習
Kneighbor.fit(features_train,label_train)
 
#K近傍法でアヤメの種類を予測
# label_pred_KNeighbor = Kneighbor.predict(features_test)
# print(label_pred_KNeighbor)

print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_KNeighbor))
print('accuracy = ', Kneighbor.score(features_test,label_test))

confusion matrix = 
 [[4038  328]
 [ 332   29]]


AttributeError: 'Flags' object has no attribute 'c_contiguous'

In [86]:
#ロジスティック回帰のインスタンスを作成
LogReg = LogisticRegression(random_state=0)
#ロジスティック回帰でデータを学習
LogReg.fit(features_train,label_train)
#ロジスティック回帰でアヤメの種類を予測
label_pred_LogReg = LogReg.predict(features_test)
print(label_pred_LogReg)

print('confusion matrix = \n', confusion_matrix(y_true = label_test, y_pred = label_pred_LogReg))
print('accuracy = ', LogReg.score(features_test,label_test))

[0 0 0 ... 0 0 0]
confusion matrix = 
 [[1511    0]
 [ 111    0]]
accuracy =  0.9315659679408138


In [87]:
model = LogisticRegression(penalty='l2',          # 正則化項(L1正則化 or L2正則化が選択可能)
                           dual=False,            # Dual or primal
                           tol=0.00001,            # 計算を停止するための基準値
                           C=1.0,                 # 正則化の強さ
                           fit_intercept=True,    # バイアス項の計算要否
                           intercept_scaling=1,   # solver=‘liblinear’の際に有効なスケーリング基準値
                           class_weight=None,     # クラスに付与された重み
                           random_state=None,     # 乱数シード
                           solver='lbfgs',        # ハイパーパラメータ探索アルゴリズム
                           max_iter=1000,          # 最大イテレーション数
                           multi_class='auto',    # クラスラベルの分類問題（2値問題の場合'auto'を指定）
                           verbose=0,             # liblinearおよびlbfgsがsolverに指定されている場合、冗長性のためにverboseを任意の正の数に設定
                           warm_start=False,      # Trueの場合、モデル学習の初期化に前の呼出情報を利用
                           n_jobs=None,           # 学習時に並列して動かすスレッドの数
                           l1_ratio=None          # L1/L2正則化比率(penaltyでElastic Netを指定した場合のみ)
                          )

model.fit(features_train,label_train)

LogisticRegression(max_iter=1000, tol=1e-05)

In [88]:
df_model = pd.DataFrame(index=['1号艇1着率','1号艇2着率','1号艇3着率','2号艇1着率','2号艇2着率','2号艇3着率','3号艇1着率','3号艇2着率','3号艇3着率','4号艇1着率','4号艇2着率','4号艇3着率','5号艇1着率','5号艇2着率','5号艇3着率','6号艇1着率','6号艇2着率','6号艇3着率'])
df_model["偏回帰係数"] = model.coef_[0]
df_model

,偏回帰係数
1号艇1着率,0.420205
1号艇2着率,0.047577
1号艇3着率,0.023195
2号艇1着率,0.224292
2号艇2着率,0.145171
2号艇3着率,0.097199
3号艇1着率,0.056498
3号艇2着率,0.106084
3号艇3着率,0.142351
4号艇1着率,-0.135989


In [108]:
pred_proba = model.predict_proba(features_test)
label_test = label_test.reset_index(drop=True)
df_proba = pd.DataFrame()
df_proba["123"] = pred_proba[:,1]
df_proba["正解"] = label_test
df_proba["レースID"] = dataset_test["レースID"]
df_proba["オッズ"] = ""
df_proba["日付"] = dataset_test["日付"].astype(int).astype(str)
df_proba[(df_proba["123"]>0.11)&(df_proba["正解"]==1)]

,123,正解,レースID,オッズ,日付
39,0.146176,1,202308011511,,20230801
94,0.120812,1,202308010107,,20230801
168,0.118057,1,202308020912,,20230802
261,0.121348,1,202308031011,,20230803
270,0.136877,1,202308030908,,20230803
355,0.111674,1,202308041208,,20230804
446,0.134876,1,202308051710,,20230805
465,0.130205,1,202308051102,,20230805
559,0.115773,1,202308061509,,20230806
577,0.110421,1,202308061111,,20230806


In [107]:
count = 0
odd = 0
pre_date = ""
od = pd.DataFrame(columns=['オッズ', 'レースID'])
for index, row in df_proba.iterrows():
    if row["日付"] != pre_date:
        odds = pd.read_csv("/mounted/tobe/odds/t" + row["日付"] + ".csv")
        odds["レースID"] = [str(v["日付"]) + str(race_place[v["開催地"]]).zfill(2) + str(v["ラウンド名"]).zfill(2) for i, v in odds.iterrows()]
    # tmp = {"オッズ":[odds[odds["レースID"] == row["レースID"]]["123"].astype(str).iloc[-1]] , "レースID":[row["レースID"]]}
    # pd.concat([od,tmp])
    df_proba.loc[index,"オッズ"] = odds[odds["レースID"] == row["レースID"]]["123"].astype(str).iloc[-1]
    if row["123"] > 0.11:
        count = count + 1
    if (row["123"] > 0.11) & (row["正解"] == 1):
        odd = odd + odds[odds["レースID"] == row["レースID"]]["123"].astype(float).iloc[-1]
    pre_date = row["日付"]
df_proba

,123,正解,レースID,オッズ,日付
0,0.164947,0,202308012302,7.1,20230801
1,0.074311,0,202308012304,20.0,20230801
2,0.047369,0,202308012307,15.7,20230801
3,0.011069,0,202308012308,56.4,20230801
4,0.054240,0,202308012309,21.3,20230801
...,...,...,...,...,...
1617,0.110450,0,202308160106,9.3,20230816
1618,0.113846,0,202308160107,7.2,20230816
1619,0.076253,1,202308160109,12.4,20230816
1620,0.080089,0,202308160111,12.9,20230816
